# Robotic Systems II (ECE-DK904)

## Electrical and Computer Engineering Department, University of Patras, Greece

**Instructor:** Konstantinos Chatzilygeroudis (costashatz@upatras.gr)

## Lab 3

### Constrained Optimization with Equality Constraints

In constrained optimization with equality constraints we need to solve the following problem:

$\min_{\boldsymbol{x}} f(\boldsymbol{x})$

$\quad\quad\text{s.t. }h(\boldsymbol{x}) = 0$

where $f(\boldsymbol{x}) : \mathbb{R}^N\to\mathbb{R}, h(\boldsymbol{x}): \mathbb{R}^N\to\mathbb{R}^M$.

We saw in the class that we can tackle this problem by defining a new function, called the `Lagrangian`:

$\mathcal{L}(\boldsymbol{x}, \boldsymbol{\lambda}) = f(\boldsymbol{x}) + \boldsymbol{\lambda}^Th(\boldsymbol{x})$

where $\boldsymbol{\lambda}\in\mathbb{R}^M$. Taking the derivative of the Langragian, we get:

$\nabla_{\boldsymbol{x}}\mathcal{L} = \nabla_{\boldsymbol{x}}f(\boldsymbol{x}) + \Big(\frac{\partial h}{\partial\boldsymbol{x}}\Big)^T\boldsymbol{\lambda} = 0$

$\nabla_{\boldsymbol{\lambda}}\mathcal{L} = h(\boldsymbol{x}) = 0$

which we call the **KKT** conditions. In other words, **these two conditions necessarily have to hold at the optimum!**

So how do we proceed in optimizing this? The main idea is to use Newton's method on the Lagrangian! In order to do so, we need to linearize the gradient of the Lagrangian around the current estimate $(\boldsymbol{x}_k,\boldsymbol{\lambda}_k)$:

$\nabla_{\boldsymbol{x}}\mathcal{L}(\boldsymbol{x}_k + \Delta\boldsymbol{x}, \boldsymbol{\lambda}_k + \Delta\boldsymbol{\lambda}) = \nabla_{\boldsymbol{x}}\mathcal{L}(\boldsymbol{x}_k, \boldsymbol{\lambda}_k) + \frac{\partial^2\mathcal{L}}{\partial\boldsymbol{x}^2}\Big|_{\boldsymbol{x}_k,\boldsymbol{\lambda}_k}\Delta\boldsymbol{x} + \underbrace{\frac{\partial^2\mathcal{L}}{\partial\boldsymbol{x}\partial\boldsymbol{\lambda}}\Big|_{\boldsymbol{x}_k,\boldsymbol{\lambda}_k}}_{\Big(\frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k}\Big)^T}\Delta\boldsymbol{\lambda} = 0$

$\nabla_{\boldsymbol{\lambda}}\mathcal{L}(\boldsymbol{x}_k + \Delta\boldsymbol{x}, \boldsymbol{\lambda}_k + \Delta\boldsymbol{\lambda}) = h(\boldsymbol{x}_k) + \frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k}\Delta\boldsymbol{x} = 0$

Solving the above we end up with the following equations:

$\begin{bmatrix}\frac{\partial^2\mathcal{L}}{\partial\boldsymbol{x}^2}\Big|_{\boldsymbol{x}_k,\boldsymbol{\lambda}_k} & \Big(\frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k}\Big)^T\\\frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k} & \boldsymbol{0}\end{bmatrix}
\begin{bmatrix}\Delta\boldsymbol{x}\\\Delta\boldsymbol{\lambda}\end{bmatrix} = 
\begin{bmatrix}-\nabla_{\boldsymbol{x}}\mathcal{L}(\boldsymbol{x}_k, \boldsymbol{\lambda}_k)\\-h(\boldsymbol{x}_k)\end{bmatrix}$

We call this the **KKT System**!

This is basically one **big linear system**. We can solve with ONE `np.linalg.solve()` call!!

**Important detail:**

$\frac{\partial^2\mathcal{L}}{\partial\boldsymbol{x}^2} = \nabla^2_{\boldsymbol{x}}f + \frac{\partial}{\partial\boldsymbol{x}}\Big[(\frac{\partial h}{\partial\boldsymbol{x}})^T\boldsymbol{\lambda}\Big]$

We usually drop the term $\frac{\partial}{\partial\boldsymbol{x}}\Big[(\frac{\partial h}{\partial\boldsymbol{x}})^T\boldsymbol{\lambda}\Big]$ since in the general case this is a 3D tensor and it's difficult to compute!

**ENOUGH TALK!** Let's optimize something:

In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
# We will define a quadratic objective function and a quadratic constraint
Q = np.array(([[0.5, 0.], [0., 1.]]))
x_target = np.array([[1.], [0.]])

def f(x):
    x = np.asarray(x).reshape((2,-1))
    return (0.5*(x - x_target).T @ Q @ (x - x_target))[0, 0]

def df(x):
    x = np.asarray(x).reshape((2,-1))
    return (x - x_target).T @ Q

def ddf(x):
    return Q

In [ ]:
def cons(x):
    x = np.asarray(x).reshape((2,-1))
    return x[0, 0]**2 + 2. * x[0, 0] - x[1, 0]

def dc(x):
    x = np.asarray(x).reshape((2,-1))
    return np.asarray([2. * x[0, 0] + 2., -1.]).reshape((1, 2))

def ddc(x, l = np.ones((2, 1))):
    l = np.asarray(l).reshape((1,-1))
    h = np.zeros((2, 2))
    h[0, 0] = 2. * l[0,0]
    return h

In [ ]:
# Newton's Method with Equality Constraints
def constrained_newton_step(x0, l0):
    N = 2
    M = 1
    x0 = np.asarray(x0).reshape((N, -1))
    l0 = np.asarray(l0).reshape((M, -1))

    # Let's compute the KKT system
    ### TO-DO: Compute the KKT matrix
    ddL_ddx = ddf(x0)
    dh_dx = dc(x0)
    KKT_system = np.block([[ddL_ddx, dh_dx.T], [dh_dx, 0]])
    ### END of TO-DO

    # Let's compute the target vector
    ### TO-DO: Compute the target vector of the KKT system
    VxL = df(x0).T + dc(x0).T @ l0
    h = cons(x0)
    KKT_target_vector = np.block([[-VxL], [-h]])
    ### END of TO-DO

    # Let's solve for Δz
    ### TO-DO: Solve for Δz
    DeltaZ = np.linalg.solve(KKT_system, KKT_target_vector)
    ### END of TO-DO

    # Decompose delta to Δx and Δλ
    ### TO-DO: Decompose delta to Δx and Δλ
    DeltaX = DeltaZ[:2]
    DeltaL = DeltaZ[2]
    ### END of TO-DO

    return x0 + DeltaX, l0 + DeltaL

In [ ]:
# Let's plot things
plt.close() # close previous
fig = plt.figure()
ax = fig.add_subplot(111)

N = 40
x1 = np.linspace(-4., 4., N)
x2 = np.linspace(-4., 4., N)

X, Y = np.meshgrid(x1, x2)

val = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        xx = np.zeros((2, 1))
        xx[0] = X[i, j]
        xx[1] = Y[i, j]
        val[i, j] = f(xx)

ax.contour(x1.reshape((N,)), x2.reshape((N,)), val, 50)

x = np.linspace(-3.2, 1.2, 1000)
y = x**2 + 2. * x
ax.plot(x, y, label='c(x)', color='orange')

In [ ]:
# Let's start optimizing
# Initial point!
x_init = np.array([[-1., -1.]]).T
l_init = np.array([[0.]])
ax.plot(x_init[0], x_init[1], 'rx')

# Optimization
x_new = np.copy(x_init)
l_new = np.copy(l_init)

fig # show figure again with updated point(s)

In [ ]:
### TO-DO: Write the iterate (aka one step) of Newton method for equality constraints
x_new, l_new = constrained_newton_step(x_new, l_new)
### END of TO-DO

ax.plot(x_new[0], x_new[1], 'rx')

fig # show figure again with updated point(s)

Nice! We can optimize nicely now! With **constraints**! Let's try another initial point:

In [ ]:
# Let's plot things
plt.close() # close previous
fig = plt.figure()
ax = fig.add_subplot(111)

N = 40
x1 = np.linspace(-4., 4., N)
x2 = np.linspace(-4., 4., N)

X, Y = np.meshgrid(x1, x2)

val = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        xx = np.zeros((2, 1))
        xx[0] = X[i, j]
        xx[1] = Y[i, j]
        val[i, j] = f(xx)

ax.contour(x1.reshape((N,)), x2.reshape((N,)), val)

x = np.linspace(-3.2, 1.2, 1000)
y = x**2 + 2. * x
ax.plot(x, y, label='c(x)', color='orange')

In [ ]:
# Different initial point!
x_init = np.array([[-3., 2.]]).T
l_init = np.array([[0.]])
ax.plot(x_init[0], x_init[1], 'rx')

# Optimization
x_new = np.copy(x_init)
l_new = np.copy(l_init)

fig # show figure again with updated point(s)

In [ ]:
### TO-DO: COPY the iterate (aka one step) of Newton method for equality constraints
x_new, l_new = constrained_newton_step(x_new, l_new)
### END of TO-DO

ax.plot(x_new[0], x_new[1], 'rx')

fig # show figure again with updated point(s)

The method is stuck! It cannot converge!! What can we do? Let's add damping/regularization! Remember our talk about *Duality*? We need the **KKT** matrix to have $N$ (dim of $\boldsymbol{x}$) positive eigenvalues and $M$ (dim of $\boldsymbol{\lambda}$) negative ones! This matrix is called **quasi-definite**. So we add regularization accordingly:

$\begin{bmatrix}\frac{\partial^2\mathcal{L}}{\partial\boldsymbol{x}^2}\Big|_{\boldsymbol{x},\boldsymbol{\lambda}} + \beta\boldsymbol{I} & \Big(\frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}}\Big)^T\\\frac{\partial h}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}} & -\beta\boldsymbol{I}\end{bmatrix}
\begin{bmatrix}\Delta\boldsymbol{x}\\\Delta\boldsymbol{\lambda}\end{bmatrix} = \begin{bmatrix}-\nabla_{\boldsymbol{x}}\mathcal{L}(\boldsymbol{x}, \boldsymbol{\lambda})\\-h(\boldsymbol{x})\end{bmatrix}$

Let's write this:

In [ ]:
# Newton's Method with Equality Constraints with Regularization
def constrained_newton_step(x0, l0, beta = 1.):
    N = 2
    M = 1
    x0 = np.asarray(x0).reshape((N, -1))
    l0 = np.asarray(l0).reshape((M, -1))

    # Let's compute the KKT system
    ### TO-DO: Copy the KKT matrix implementation from above
    ddL_ddx = ddf(x0)
    dh_dx = dc(x0)
    KKT_system = np.block([[ddL_ddx, dh_dx.T], [dh_dx, 0]])
    ### END of TO-DO

    ### TO-DO: Implement the above regularization!
    KKT_system[:2,:2] += beta * np.eye(2)
    KKT_system[2,2] -= beta * 1

    eig = np.linalg.eigvals(KKT_system)
    pos_count = 0
    neg_count = 0
    zer_count = 0
    for e in eig:
        if(e > 0):
            pos_count += 1 
        if(e < 0):
            neg_count += 1
            
    while(pos_count != 2 and neg_count != 1):
        KKT_system[:2,:2] += beta * np.eye(2)
        KKT_system[2,2] -= beta * 1
        
        eig = np.linalg.eigvals(KKT_system)
        
        pos_count = 0
        neg_count = 0
        zer_count = 0
        for e in eig:
            if(e > 0):
                pos_count += 1
            if(e < 0):
                neg_count += 1
    ### END of TO-DO

    # Let's compute the target vector
    ### TO-DO: Copy the target vector of the KKT system from above
    VxL = df(x0).T + dc(x0).T @ l0
    h = cons(x0)
    KKT_target_vector = np.block([[-VxL], [-h]])
    ### END of TO-DO

    # Let's solve for Δz
    ### TO-DO: Solve for Δz
    DeltaZ = np.linalg.solve(KKT_system, KKT_target_vector)
    ### END of TO-DO

    # Decompose delta to Δx and Δλ
    ### TO-DO: Decompose delta to Δx and Δλ
    DeltaX = DeltaZ[:2]
    DeltaL = DeltaZ[2]
    ### END of TO-DO

    return x0 + DeltaX, l0 + DeltaL

In [ ]:
# Let's try again!
plt.close() # close previous
fig = plt.figure()
ax = fig.add_subplot(111)

N = 40
x1 = np.linspace(-4., 4., N)
x2 = np.linspace(-4., 4., N)

X, Y = np.meshgrid(x1, x2)

val = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        xx = np.zeros((2, 1))
        xx[0] = X[i, j]
        xx[1] = Y[i, j]
        val[i, j] = f(xx)

ax.contour(x1.reshape((N,)), x2.reshape((N,)), val, 50)

x = np.linspace(-3.2, 1.2, 1000)
y = x**2 + 2. * x
ax.plot(x, y, label='c(x)', color='orange')

# Different initial point!
x_init = np.array([[-3., 2.]]).T
l_init = np.array([[0.]])
ax.plot(x_init[0], x_init[1], 'rx')

# Optimization
x_new = np.copy(x_init)
l_new = np.copy(l_init)

In [ ]:
### TO-DO: COPY the iterate (aka one step) of Newton method for equality constraints
x_new, l_new = constrained_newton_step(x_new, l_new)
### END of TO-DO

ax.plot(x_new[0], x_new[1], 'rx')

print(f(x_new))

fig # show figure again with updated point(s)

This is much better!

**Re-try the above but remove the term involving the second derivative of the constraints!** Aka, try the **Gauss-Newton** method.

You see that it is more robust! This is because even if the Hessian of $f()$ is well-behaved, the term involving the second derivative of the constraints can *spoil the soup*! And it is also usually **expensive to compute**. For these reasons, we usually drop this completely!

### Constrained Optimization with Inequality Constraints

In constrained optimization with inequality constraints we need to solve the following problem:

$\min_{\boldsymbol{x}} f(\boldsymbol{x})$

$\quad\quad\text{s.t. }g(\boldsymbol{x})\leq 0$

where $f(\boldsymbol{x}) : \mathbb{R}^N\to\mathbb{R}, g(\boldsymbol{x}): \mathbb{R}^N\to\mathbb{R}^M$.

The **Lagrangian** of this is:

$\mathcal{L}(\boldsymbol{x}, \boldsymbol{\mu}) = f(\boldsymbol{x}) + \boldsymbol{\mu}^Tg(\boldsymbol{x})$

In this version, we cannot apply the tricks that we did before because we do not have a root finding problem for the constraints!

#### Augmented Lagrangian Method

We saw in the lectures **Augmented Lagrangian Method**, where we define a new function called **Augmented Lagrangian**:

$\mathcal{L}_{\rho}(\boldsymbol{x}, \boldsymbol{\mu}) = f(\boldsymbol{x}) + \boldsymbol{\mu}^Tg(\boldsymbol{x}) + \frac{\rho}{2}\lVert\text{max}(0, g(\boldsymbol{x}))\rVert^2$

So we define a procedure as follows:

1) $\boldsymbol{x}_{k+1} = \min_{\boldsymbol{x}}\mathcal{L}_{\rho}(\boldsymbol{x}, \boldsymbol{\mu}_k)$
2) $\boldsymbol{\mu}_{k+1} = \text{max}\Big(\boldsymbol{0}, \boldsymbol{\mu}_k + \rho g(\boldsymbol{x}_{k+1})\Big)$
3) $\rho = \alpha\rho,\text{ with }\alpha\in\mathbb{R}^{+}$

We solve the minimization in step (1) with Newton's method since this is unconstrained minimization in $\boldsymbol{x}$ (we keep $\boldsymbol{\mu}_k$ fixed!). We usually use the **Gauss-Newton** version and:

$\Delta\boldsymbol{x} = -\Big(\nabla^2_{\boldsymbol{x}}f(\boldsymbol{x}_k) + \rho\Big(\frac{\partial g}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k}\Big)^T\frac{\partial g}{\partial\boldsymbol{x}}\Big|_{\boldsymbol{x}_k}\Big)^{-1}\nabla_{\boldsymbol{x}}\mathcal{L}_{\rho}(\boldsymbol{x}_k, \boldsymbol{\mu}_k)$

Let's implement it? Let's first implement a function that computes the augmented Lagrangian (it will not be actually useful, but it will help your understanding):

In [ ]:
def La(x, mu, rho):
    ### TO-DO: Compute the Augmented Lagrangian
    return f(x) + mu.T @ cons(x) + (rho/2) * np.linalg.norm(np.maximum(0, cons(x)))**2
    ### END of TO-DO

Now we need to implement step (1) of the Augmented Lagrangian Method, that is the minimization of the augmented Lagrangian over $\boldsymbol{x}$ alone. Let's do this with Newton's method:

In [ ]:
# Here's the optimization routine for La(x,mu,rho) with mu, rho fixed! Running Newton to find the minimum!
def newton_solve(x0, mu, rho):
    x = np.copy(x0)
    x = np.asarray(x).reshape((2,-1))

    ### TO-DO: Compute the residual, i.e. d(Lα)/dx
    r = (df(x) + mu.T @ dc(x) + rho * np.linalg.norm(np.maximum(0, cons(x))) * dc(x)).T
    ### END of TO-DO
    while np.linalg.norm(r) >= 1e-8:
        ### TO-DO: Compute Hessian of La (Gauss-Newton, we skip the second derivatives of the constraints)
        H = ddf(x) + rho * (dc(x).T @ dc(x)) 
        ### END of TO-DO

        # Compute Δx
        DeltaX = -np.linalg.solve(H, r)

        # Step iterate x_k
        x = x + DeltaX

        ### TO-DO: Recompute the residual, i.e. d(Lα)/dx (just copy the above)
        r = (df(x) + mu.T @ dc(x) + rho * np.linalg.norm(np.maximum(0, cons(x))) * dc(x)).T
        ### END of TO-DO
    return x

Let's see what it does in our test function. Now we interpret the constraints as $g(\boldsymbol{x})\leq 0$ instead of equality ones.

In [ ]:
# Let's plot things
plt.close() # close previous
fig = plt.figure()
ax = fig.add_subplot(111)

N = 40
x1 = np.linspace(-4., 4., N)
x2 = np.linspace(-4., 4., N)

X, Y = np.meshgrid(x1, x2)

val = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        xx = np.zeros((2, 1))
        xx[0] = X[i, j]
        xx[1] = Y[i, j]
        val[i, j] = f(xx)

ax.contour(x1.reshape((N,)), x2.reshape((N,)), val)

x = np.linspace(-3.2, 1.2, 1000)
y = x**2 + 2. * x
ax.plot(x, y, label='c(x)', color='orange')

In [ ]:
# Initial point!
x_init = np.array([[-3., 2.]]).T
mu_init = np.array([[0.]])
rho = 1.
ax.plot(x_init[0], x_init[1], 'rx')

# Optimization
x_new = np.copy(x_init)
mu_new = np.copy(mu_init)

fig # show figure again with updated point(s)

In [ ]:
### TO-DO: Implement step (1), (2), and (3) of ALM
alpha = 10.
x_new = newton_solve(x_new, mu_new, rho)
mu_new = np.maximum(np.array([[0.]]), mu_new + rho * cons(x_new))
rho = alpha * rho
### END of TO-DO

ax.plot(x_new[0], x_new[1], 'rx')

print(cons(x_new))
print(f(x_new))

fig # show figure again with updated point(s)

### Quadratic Programming

A special case of constrained optimization that is very useful in control and robotics is the **Quadratric Programming** (QP) problem:

$\min_{\boldsymbol{x}}f(\boldsymbol{x}) = \frac{1}{2}\boldsymbol{x}^T\boldsymbol{Q}\boldsymbol{x} + \boldsymbol{q}^T\boldsymbol{x}$

$\quad\quad\text{s.t. }~~\boldsymbol{A}\boldsymbol{x}-\boldsymbol{b} = \boldsymbol{0}$

$\quad\quad\quad\quad\boldsymbol{C}\boldsymbol{x}-\boldsymbol{d}\leq\boldsymbol{0}$

where $\boldsymbol{x},\boldsymbol{q}\in\mathbb{R}^N$, $\boldsymbol{Q}\succ 0\in\mathbb{R}^{N\times N}$.

We can solve this special case algorithm with the **Augmented Lagrangian**, but there have been developed many specific methods that optimally exploit the specific structure of the problem. There are many libraries available to use. We will use the [ProxSuite](https://github.com/Simple-Robotics/proxsuite) one that uses a state of the art **proximal method** (based on the Augmented Lagrangian) to solve QP problems.

*ProxSuite* solves problems of the form:

$\min_{\boldsymbol{x}}f(\boldsymbol{x}) = \frac{1}{2}\boldsymbol{x}^T\boldsymbol{Q}\boldsymbol{x} + \boldsymbol{q}^T\boldsymbol{x}$

$\quad\quad\text{s.t. }~~\boldsymbol{A}\boldsymbol{x}-\boldsymbol{b} = \boldsymbol{0}$

$\quad\quad\quad\quad\boldsymbol{l}\leq\boldsymbol{C}\boldsymbol{x}\leq\boldsymbol{u}$

So let's define a QP problem and try to solve it with the *ProxSuite* library. First, let's import the library:

In [ ]:
import proxsuite

Now let's generate a random QP problem:

In [ ]:
# Generate a random non-trivial quadratic program.
n = 5 # number of dimensions of the optimization variables (x)
m = 2 # number of inequality constraints
p = 1 # number of equality constraints

# Random Cost
Q = np.random.randn(n, n)
Q = Q.T @ Q # We need to make sure that Q is positive definite
q = np.random.randn(n)

# Random inequality constraints
C = np.random.randn(m, n)
u = C @ np.random.randn(n)

# Random equality constraints
A = np.random.randn(p, n)
b = np.random.randn(p)

In [ ]:
# Let's create the solver
qp_dim = Q.shape[0]
qp_dim_eq = A.shape[0]
qp_dim_in = C.shape[0]
qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

# initialize the model of the problem to solve
qp.init(Q, q, A, b, C, u, None)
qp.solve()

# Get the result
print("optimal x: {}".format(qp.results.x))

So now we can easily define QP problems, give them to *ProxSuite* and get back the results! Easy!

### General Non-Linear Programming (NLP)

What if we have a general non linear problem with constraints that we want to minimize/optimize? We can implement the **Augmented Lagrangian Method** but in practice many small tricks are needed to have a robust solver. So what should we do? Luckily, there are quite a few good libraries out there for solving generic NLP problems. In this course, we will use [Ipopt](https://github.com/coin-or/Ipopt).

Ipopt solves generic NLP problems with any type of objective functions and constraints. We need to provide to Ipopt:

1) The objective function and its gradient
2) The constraints and their Jacobians
3) Bounds to the variables

Then Ipopt does all the job and we get back a nice result (most of the times!). In the backed, Ipopt uses a **Primal-Dual Interior Point Method**.

So let's see it in action! First we import the Python bindings of the library.

In [ ]:
import cyipopt

Let's optimize the following function:

$f(\boldsymbol{x}) = (x_1-2)^2 + (x_2-1)^2$

with the following constraints:

$h(\boldsymbol{x}) = x_1-2\,x_2+1=0$

$g(\boldsymbol{x}) = 0.25\,{x_1^2}+x_2^2-1\leqslant 0$

The best known optimal feasible solution is $f(\boldsymbol{x^*}) = 1.3934651$.

In [ ]:
# So here's how to write this for cyipopt
class IpoptOptimization:
    def __init__(self):
        pass

    def objective(self, x):
        return (x[0] - 2.)**2 + (x[1]-1.)**2

    def gradient(self, x):
        return np.asarray([2. * (x[0] - 2.), 2. * (x[1] - 1.)])

    def constraints(self, x):
        c = np.zeros((2,))

        # Equality
        c[0] = x[0] - 2. * x[1] + 1.
        # Inequality
        c[1] = 0.25 * x[0] * x[0] + x[1] * x[1] - 1.

        return c

    def jacobian(self, x):
        J = np.zeros((2, 2))
        # Gradient of 1st constraint wrt to x1
        J[0, 0] = 1.
        # Gradient of 1st constraint wrt to x2
        J[0, 1] = -2.

        # Gradient of 2nd constraint wrt to x1
        J[1, 0] = 0.5 * x[0]
        # Gradient of 2nd constraint wrt to x2
        J[1, 1] = 2. * x[1]
        
        return J

# Let's give a zero initial guess
x0 = np.zeros((2,))

# Bounds for constraints
cl = np.zeros((2,))
cl[1] = -np.inf # Inequality has no lower bound

cu = np.zeros((2,))

# Let's solve the problem
nlp = cyipopt.Problem(n=2, m=2, problem_obj=IpoptOptimization(), lb=[None]*2, ub=[None]*2, cl=cl, cu=cu)

nlp.add_option("jacobian_approximation", "exact") # "finite-difference-values")
nlp.add_option("print_level", 3)
nlp.add_option("nlp_scaling_method", "none")


# Solve the problem
x, info = nlp.solve(x0)

# Optimal
print(x)
print(info['obj_val'])